In [31]:
import numpy as np
import pandas as pd

### Este notebook auna todo el preprocesado necesario para transformar los datos de la carpeta RAW a PREPARED:

In [32]:
path_in  = "../raw/trabajo6.csv"
path_out = "../partitioned/"

In [33]:
df = pd.read_csv(path_in)

### One Hot Encoding

In [34]:
df = df.join(pd.get_dummies(df['type']))

### Nuevas variables

In [35]:
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']

mean_oldOrg  = df['oldbalanceOrg'].mean()
mean_oldDest = df['oldbalanceDest'].mean()

# Para evitar NaN. No afecta la distribución de los datos.
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

### Particionamos el dataset en 12 partes:

In [36]:
df['partition'] = np.random.randint(12, size=len(df))

In [37]:
df.partition.value_counts()

3     531487
1     531019
0     530594
8     530453
7     530403
2     530290
11    529943
6     529904
5     529809
4     529762
10    529683
9     529273
Name: partition, dtype: int64

### Write Prepared

In [38]:
# Necesita paquete 'pyarrow'

In [39]:
df.to_parquet(path_out, partition_cols='partition', compression='snappy')

In [24]:
# Para leer el parquet:
p_df = pd.read_parquet(path_out).drop(columns=['partition'])

In [41]:
df.type.unique().tolist()

['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN']

In [42]:
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,diffOrigen,diffDestino,cambioOrigen,cambioDestino,partition
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,...,0,0,0,1,0,-9839.64,0.00,-0.057496,0.000010,6
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,...,0,0,0,1,0,-1864.28,0.00,-0.083791,0.000010,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,...,0,0,0,0,1,-181.00,0.00,-0.153251,0.000010,6
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,...,0,1,0,0,0,-181.00,-21182.00,-0.153251,-0.954918,11
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,...,0,0,0,1,0,-11668.14,0.00,-0.274196,0.000010,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,...,0,1,0,0,0,-339682.13,339682.13,-0.997065,339.682140,10
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,...,0,0,0,0,1,-6311409.28,0.00,-0.999842,0.000010,5
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,...,0,1,0,0,0,-6311409.28,6311409.27,-0.999842,90.826229,9
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,...,0,0,0,0,1,-850002.52,0.00,-0.998825,0.000010,4


In [ ]:
'amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','cambioOrigen','cambioDestino'